In [21]:
import einops
from torch import nn
import torch
import pickle
import sys
sys.path.append('../')
from models import misc_mnet as mnet



class Meta_Mnet(nn.Module):
    def __init__(self, batchsize=32, dim_a=16, mode='glasso',
                 normalize=False, beta=0, temperature=1.0, **kwargs):
        super().__init__()

        #EINOPS CAUSES A PROBLEM!!!
        #self.Ms = nn.Parameter(einops.repeat(torch.eye(dim_a), 'c a -> b c a', b = batchsize).float())
        
        
        #self.Ms = nn.Parameter(torch.tensor(np.random.normal(size=(32, 16, 16))).float())
        self.Ms = nn.Parameter(torch.stack([torch.eye(16)]*32))


        # torch.nn.init.orthogonal_(self.U)
        self.mode = mode

        self.Mnet_args = {'normalize':normalize,
                          'dim_a': dim_a,
                          'beta': beta,
                          'temperature':temperature}

    def __call__(self, H0, H1):
        H1hat = H0 @ self.Ms

        if self.mode == 'exact':
            loss = loss_l2_norm(H1hat, H1)                                       
        elif self.mode == 'glasso':
            loss = mnet.loss_group_lasso_norm(H1hat, H1)
        else:
            
            raise NotImplementedError

        return loss

    
def loss_l2_norm(z1, z2, **kwargs):
    return torch.sqrt(torch.sum((z1 -  z2) **2))

In [24]:
inner_args = {'normalize': 0, 'batchsize': 32, 'num_loops': 20, 'detach': 1, 'beta': 0, 'temperature': 1, 'mode': 'exact', 'lr': 0.01, 'dim_a': 16}

with open('H0H1.pkl', 'rb') as fp:
    problem = pickle.load(fp)
H0 = problem['H0']
H1 = problem['H1']
print(H0.shape)
import numpy as np


np.random.seed(1)
trueM = torch.tensor(np.random.normal(size = (H0.shape[0], 16, 16))).float()
trueH1 = H0 @ trueM
inner_args['num_loops']=20
inner_args['lr']=0.1  


Mobj = Meta_Mnet(dim_a=16, batchsize=32, mode='glasso')
optimizer = torch.optim.SGD(Mobj.parameters(), inner_args['lr'])

for k in range(inner_args['num_loops']):
    optimizer.zero_grad()
    loss = Mobj(H0, trueH1)
    print(f"""inner_loss {loss.item()}""")
    loss.backward()
    optimizer.step()

torch.Size([32, 256, 16])
inner_loss 18.008941650390625
inner_loss 17.99810791015625
inner_loss 17.9870548248291
inner_loss 17.975772857666016
inner_loss 17.96426010131836
inner_loss 17.952512741088867
inner_loss 17.94052505493164
inner_loss 17.92829704284668
inner_loss 17.915817260742188
inner_loss 17.903091430664062
inner_loss 17.89011001586914
inner_loss 17.87687110900879
inner_loss 17.863378524780273
inner_loss 17.84962272644043
inner_loss 17.835603713989258
inner_loss 17.821317672729492
inner_loss 17.8067684173584
inner_loss 17.791948318481445
inner_loss 17.77686309814453
inner_loss 17.761505126953125


In [3]:
print(Mobj.Ms.shape)

torch.Size([32, 16, 16])


In [4]:
print(H0.shape)

torch.Size([32, 256, 16])


In [5]:
(H0 @ Mobj.Ms).shape

torch.Size([32, 256, 16])

In [6]:
H0.device, Mobj.Ms.device

(device(type='cpu'), device(type='cpu'))